<img src='https://repository-images.githubusercontent.com/121802384/c355bb80-7d42-11e9-9e0e-4729609f9fbc' alt='WRF-Hydro Logo' width="15%"/>

# Lesson S2- Regridding supplemental precip forcing data to the WRF-Hydro domain

## Overview
In this lesson, we cover regridding Multi Radar Multi Sensor(MRMS) forcing data to a WRF-Hydro domain using the Python-based WRF-Hydro Forcing Engine. 

## Required software and datasets

### Meteorological Forcing Engine (MFE) software
The WRF-Hydro Meteorological Forcing Engine (MFE) software can be downloaded from https://github.com/NCAR/WrfHydroForcing

### Data sources
For this training, data has already been included for the simulation time period. 

**NOTE: These data are for instructional purposes only and do not include forcing data used for model spinup**


**Step 1: Extract the NLDAS2 forcing data provided with this tutorial**

In [ ]:
%%bash
cd ~/wrf-hydro-training/example_case/regridding
tar -xf fe_forcing.tar.gz

**Step 2: View the contents of the input_files directory**

In [ ]:
%%bash
ls ~/wrf-hydro-training/example_case/regridding/fe_forcing/supp_netcdf/20230720 | head -10

Let's view the data in the MRMS supplemental precip inputs:

In [ ]:
import xarray as xr
import hvplot.xarray

ldas_in = xr.open_mfdataset('/home/docker/wrf-hydro-training/example_case/regridding/fe_forcing/supp_netcdf/20230720/*')

input_field = ldas_in.PrecipRate_0mabovemeansealevel

input_field.hvplot(groupby='time', data_aspect=1.0, xaxis=None, yaxis=None,
                   title=input_field.short_name, attr_labels=False, widget_location='bottom',
                   cmap='gist_ncar', clim=(input_field.min(), input_field.max()))

**Step 3: View the domain metadata files**

The Forcing Engine requires two additional static input files; a WRF *geogrid* file (for grid information), and an optional Spatial Metadata template file to copy attributes from.

In [ ]:
%%bash
cp /home/docker/DOMAIN/geo_em.d0x.nc /home/docker/wrf-hydro-training/example_case/regridding/fe_forcing/
cp /home/docker/DOMAIN/GEOGRID_LDASOUT_Spatial_Metadata.nc /home/docker/wrf-hydro-training/example_case/regridding/fe_forcing/
ls ~/wrf-hydro-training/example_case/regridding/fe_forcing/*nc

We have the necessary files, now we can begin the regridding process.

## Editing the forcing engine configuration file
The WRF-Hydro Forcing Engine requires a configuration file to specify the input types, forecast time periods, and other options needed for regridding and pre-processing the forcing data.

**Step 1: Examine the supplied *nldas_fe.config* configuration file and verify the times and forcing input types**

In [ ]:
%%bash
mkdir /home/docker/wrf-hydro-training/example_case/regridding/fe_forcing/output
cat ~/wrf-hydro-training/example_case/regridding/fe_forcing/fe.config

## Regridding the forcing data using the forcing engine
In this section we will regrid all our forcing data using the configuration shown above

**Step 1: Run the Forcing Engine driver (`genForcing.py`)**

`genForcing.py` takes 3 arguments: the path to the configuration file, a version number, and a configuration name. The version and name are arbitrary and used for logging and are inserted as netCDF Global Attributes in the LDASIN output files.

In [ ]:
%%bash
cd ~/wrf-hydro-training/example_case/regridding/fe_forcing
rm -rf output/*          # clean up any existing runs
python WrfHydroForcing/genForcing.py fe.config 3.0 MR > .tmp

The Forcing Engine sends most of its output to a log file specified in the configuration file. Only severe errors are printed to *stdout* so a successful run will produce no output.

**Step 2: View the Forcing Engine Log:**

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/example_case/regridding/fe_forcing/output/LOG_MR*

**Step 3: View the contents of the *output_files* directory**

The Forcing Engine will generate output files in a directory heirarchy rooted in `output_files`. Lets view the contents of the directory:

In [ ]:
%%bash
ls -R ~/wrf-hydro-training/example_case/regridding/fe_forcing/output

**Step 4: View the contents of the LDASIN files**

Now, let's look at the actual regridded data in the output files. Try changing the variable from "RAINRATE" to others in the file to see the other fields.

In [ ]:
import xarray as xr
import hvplot.xarray

ldas_in = xr.open_mfdataset('/home/docker/wrf-hydro-training/example_case/regridding/fe_forcing/output/2023072000/2023*.PRECIP_DOMAIN1')
ldas_in

In [ ]:
output_field = ldas_in.precip_rate

output_field.hvplot(groupby='time', xaxis=None, yaxis=None, 
                    title=output_field.name, attr_labels=False, widget_location='bottom',
                    cmap='gist_ncar', clim=(output_field.min(), output_field.max()))

© UCAR 2020